In [1]:
import duckdb

import numpy as np
import geopandas as gpd
import shapely
import plotly.express as px

import os

In [2]:
connection = duckdb.connect(
    database='databases/atc.db',
    config={'allow_unsigned_extensions' : 'true'}
)

connection.load_extension('../../build/release/extension/mobilityduck/mobilityduck.duckdb_extension')

[mobilityduck] Using SRID CSV: /home/ubuntu/duckdb/official/MobilityDuck/build/release/vcpkg_installed/arm64-linux/share/spatial_ref_sys.csv


# Create tables and load data

In [3]:
query = """
CREATE OR REPLACE TABLE FlightsInput(
    Et BIGINT,
    ICAO24 VARCHAR(20), 
    Lat FLOAT,
    Lon FLOAT, 
    Velocity FLOAT,
    Heading FLOAT, 
    VertRate FLOAT, 
    CallSign VARCHAR(10),
    OnGround BOOLEAN,
    Alert BOOLEAN, 
    SPI BOOLEAN, 
    Squawk INTEGER, 
    BaroAltitude NUMERIC(7,2),
    GeoAltitude NUMERIC(7,2), 
    LastPosUpdate NUMERIC(13,3),
    LastContact NUMERIC(13,3)
);
"""

connection.execute(query).fetchone()

In [4]:
filenames = [f for f in os.listdir('data/') if f.endswith('.csv')]

for idx, filename in enumerate(filenames):
    query = f"""
COPY FlightsInput FROM 'data/{filename}';
SELECT COUNT(*) FROM FlightsInput;
"""
    result = connection.execute(query).fetchone()
    print(f'[{idx+1}/{len(filenames)}] Finished loading {filename}, total rows: {result[0]}')

[1/12] Finished loading states_2020-06-01-03.csv, total rows: 554117
[2/12] Finished loading states_2020-06-01-05.csv, total rows: 1065404
[3/12] Finished loading states_2020-06-01-04.csv, total rows: 1569343
[4/12] Finished loading states_2020-06-01-00.csv, total rows: 2495888
[5/12] Finished loading states_2020-06-01-02.csv, total rows: 3156653
[6/12] Finished loading states_2020-06-01-11.csv, total rows: 4238400
[7/12] Finished loading states_2020-06-01-07.csv, total rows: 4875301
[8/12] Finished loading states_2020-06-01-06.csv, total rows: 5420001
[9/12] Finished loading states_2020-06-01-01.csv, total rows: 6232717
[10/12] Finished loading states_2020-06-01-09.csv, total rows: 7157657
[11/12] Finished loading states_2020-06-01-08.csv, total rows: 7973646
[12/12] Finished loading states_2020-06-01-10.csv, total rows: 8934621


In [5]:
query = """
ALTER TABLE FlightsInput ADD COLUMN EtTs TIMESTAMP;
ALTER TABLE FlightsInput ADD COLUMN LastPosUpdateTs TIMESTAMP;
ALTER TABLE FlightsInput ADD COLUMN LastContactTs TIMESTAMP;
UPDATE FlightsInput SET
    EtTs = to_timestamp(Et),
    LastPosUpdateTs = to_timestamp(LastPosUpdate),
    LastContactTs = to_timestamp(LastContact);

ALTER TABLE FlightsInput
ADD COLUMN Geo geometry;

UPDATE FlightsInput SET Geo = ST_Point(Lon, Lat);

WITH ICAO24_WithNullLon AS (
    SELECT ICAO24, COUNT(Lat)
    FROM FlightsInput
    GROUP BY ICAO24
    HAVING COUNT(Lon) = 0 )
DELETE FROM FlightsInput
WHERE ICAO24 IN (SELECT ICAO24 FROM ICAO24_WithNullLon);
DELETE FROM FlightsInput WHERE Squawk IS NULL;

CREATE INDEX ICAO24_time_index ON FlightsInput (ICAO24, EtTs);

SELECT COUNT(*) FROM FlightsInput;
"""

result = connection.execute(query).fetchone()
result

(5724784,)

In [6]:
query = """
CREATE OR REPLACE TABLE FlightsDay(ICAO24, Flight, Velocity, Heading, VertRate,
  CallSign, Alert, GeoAltitude) AS
SELECT ICAO24,
    tgeompointSeq(array_agg(tgeompoint(Geo::WKB_BLOB, EtTs, 4326) ORDER BY EtTs)
        FILTER (WHERE Geo IS NOT NULL)),
    tfloatSeq(array_agg(tfloat(Velocity, EtTs) ORDER BY EtTs)
        FILTER (WHERE Velocity IS NOT NULL)),
    tfloatSeq(array_agg(tfloat(Heading, EtTs) ORDER BY EtTs)
        FILTER (WHERE Heading IS NOT NULL)),
    tfloatSeq(array_agg(tfloat(VertRate, EtTs) ORDER BY EtTs)
        FILTER (WHERE VertRate IS NOT NULL)),
    ttextSeq(array_agg(ttext(CallSign, EtTs) ORDER BY EtTs)
        FILTER (WHERE CallSign IS NOT NULL)),
    tboolSeq(array_agg(tbool(Alert, EtTs) ORDER BY EtTs)
        FILTER (WHERE Alert IS NOT NULL)),
    tfloatSeq(array_agg(tfloat(GeoAltitude, EtTs) ORDER BY EtTs)
        FILTER (WHERE GeoAltitude IS NOT NULL))
FROM FlightsInput
GROUP BY ICAO24;

SELECT COUNT(*) FROM FlightsDay;
"""

result = connection.execute(query).fetchone()
result

(11736,)

In [7]:
query = """
CREATE OR REPLACE TABLE Flights(ICAO24, CallSign, FlightPeriod, Flight, Velocity, Heading, VertRate, Alert, GeoAltitude) AS
WITH Temp AS (
    SELECT ICAO24, unnest(tempDump(CallSign), recursive:=true) from FlightsDay)
SELECT f.ICAO24, value, time,
    atTime(Flight, time), atTime(Velocity, time),
    atTime(Heading, time), atTime(VertRate, time),
    atTime(Alert, time), atTime(GeoAltitude, time)
FROM FlightsDay f, Temp
WHERE f.ICAO24 = Temp.ICAO24;

SELECT COUNT(*) FROM Flights;
"""

result = connection.execute(query).fetchone()
result

(15786,)

In [8]:
query = """
SELECT ICAO24, CallSign,
    FlightPeriod::VARCHAR,
    asText(Flight),
    Velocity::VARCHAR,
    Heading::VARCHAR,
    VertRate::VARCHAR,
    Alert::VARCHAR,
    GeoAltitude::VARCHAR
FROM Flights
LIMIT 5;
"""

result = connection.execute(query).fetchdf()
result

,ICAO24,CallSign,CAST(FlightPeriod AS VARCHAR),astext(Flight),CAST(Velocity AS VARCHAR),CAST(Heading AS VARCHAR),CAST(VertRate AS VARCHAR),CAST(Alert AS VARCHAR),CAST(GeoAltitude AS VARCHAR)
0,71c334,JJA301,"{[2020-06-01 10:06:00+00, 2020-06-01 11:46:30+...",{[POINT(127.1260757446289 37.003509521484375)@...,"{[139.25001525878906@2020-06-01 10:06:00+00, 1...","{[8.070232391357422@2020-06-01 10:06:00+00, 7....","{[-4.226560115814209@2020-06-01 10:06:00+00, -...","{[f@2020-06-01 10:06:00+00, f@2020-06-01 11:46...","{[3505.2@2020-06-01 10:06:00+00, 3444.24@2020-..."
1,7817e1,CCA4337,"{[2020-06-01 01:02:50+00, 2020-06-01 07:08:40+...",{[POINT(104.88478088378906 29.722732543945312)...,"{[221.36399841308594@2020-06-01 01:02:50+00, 2...","{[165.18865966796875@2020-06-01 01:02:50+00, 1...","{[4.551680088043213@2020-06-01 01:02:50+00, 7....","{[f@2020-06-01 01:02:50+00, t@2020-06-01 03:00...","{[8420.1@2020-06-01 01:02:50+00, 8465.82@2020-..."
2,ac82bc,N905LC,"{[2020-06-01 00:00:10+00, 2020-06-01 03:39:40+...",{[POINT(-77.86571502685547 34.97314453125)@202...,"{[192.92747497558594@2020-06-01 00:00:10+00, 1...","{[0.61113178730011@2020-06-01 00:00:10+00, 0.6...","{[0@2020-06-01 00:00:10+00, 0@2020-06-01 00:00...","{[f@2020-06-01 00:00:10+00, f@2020-06-01 03:39...","{[12313.92@2020-06-01 00:00:10+00, 12306.3@202..."
3,4bb47a,MNB841,"{[2020-06-01 11:40:20+00, 2020-06-01 11:59:50+...",{[POINT(77.42523193359375 43.53788757324219)@2...,"{[177.49734497070312@2020-06-01 11:40:20+00, 1...","{[56.58627700805664@2020-06-01 11:40:20+00, 56...","{[17.231359481811523@2020-06-01 11:40:20+00, 1...","{[f@2020-06-01 11:40:20+00, f@2020-06-01 11:59...","{[3954.78@2020-06-01 11:40:20+00, 4114.8@2020-..."
4,4b191b,SWR92,"{[2020-06-01 05:22:50+00, 2020-06-01 11:17:50+...",{[POINT(8.55738194865884 47.46051848614154)@20...,"{[60.04822891167384@2020-06-01 05:22:50+00, 62...","{[319.63536992742115@2020-06-01 05:22:50+00, 3...","{[-0.09268454400893@2020-06-01 05:22:50+00, 0@...","{[t@2020-06-01 05:22:50+00, f@2020-06-01 05:23...","{[472.44@2020-06-01 05:22:50+00, 472.44@2020-0..."


# Analysis of flights

In [9]:
query = """
CREATE OR REPLACE TABLE SRID3112(Env) AS
SELECT ST_MakeEnvelope(93.41, -60.55, 173.34, -8.47)::WKB_BLOB;

CREATE OR REPLACE TABLE FlightsAust(ICAO24, CallSign, FlightPeriod, Flight, Velocity, Heading, 
    VertRate, Alert, GeoAltitude) AS
SELECT ICAO24, CallSign, FlightPeriod, transform(Flight, 3112),
    Velocity, Heading, VertRate, Alert, GeoAltitude
FROM Flights f, SRID3112 s
WHERE ST_GeometryType(trajectory(flight)::GEOMETRY) = 'LINESTRING' AND 
    eintersects(f.flight, s.Env);

SELECT COUNT(*) FROM FlightsAust;
"""

result = connection.execute(query).fetchone()
result

(873,)

Plot the trajectories of flights from `FlightsAust`

In [10]:
def plot_trajectories(df, geom_col, crs='EPSG:3112', zoom=2):
    df_new = df.copy()
    df_new['geom'] = df_new[geom_col].apply(lambda x: shapely.wkt.loads(x))
    gdf = gpd.GeoDataFrame(df_new, geometry='geom', crs=crs)
    
    df_px = gdf.copy()
    df_px = df_px.set_crs(crs)
    df_px = df_px.to_crs('EPSG:4326')

    lats = []
    lons = []

    for feature in df_px['geom']:
        if isinstance(feature, shapely.geometry.linestring.LineString):
            linestrings = [feature]
        elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
            linestrings = feature.geoms
        else:
            continue
        for linestring in linestrings:
            x, y = linestring.xy
            lats = np.append(lats, y)
            lons = np.append(lons, x)
            lats = np.append(lats, None)
            lons = np.append(lons, None)

    fig = px.line_map(lat=lats, lon=lons, map_style="carto-darkmatter", zoom=zoom)
    fig.update_layout(margin={'r': 0, 't': 0, 'l': 0, 'b': 0})

    fig.show(renderer="notebook")

    return gdf

In [11]:
query = """
SELECT CallSign,
    ST_AsText(trajectory(Flight)::GEOMETRY) AS traj
FROM FlightsAust;
"""

flightsAU = connection.execute(query).fetchdf()
flightsAU.head(2)

,CallSign,traj
0,ANZ850L,LINESTRING (3167155.5799229573 -5381697.775740...
1,SND4,LINESTRING (1509269.7538551365 -4063858.980403...


In [ ]:
_ = plot_trajectories(flightsAU, 'traj', crs='EPSG:3112')

<center><img src="figures/flights_AU.png" width=550/></center>

## Query 5.1.

Compute the flights over Australia between 8:00 am and 9:00 am on June 1st, 2020.

In [13]:
query = """
WITH Australia(AustEnv) AS (
    SELECT ST_MakeEnvelope(-1818009.6284, -5280319.0828, 2199539.9748, -1289403.7614)::WKB_BLOB),
TimePeriod(Period) AS (
    SELECT tstzspan '[2020-06-01 08:00:00, 2020-06-01 09:00:00)')
SELECT ICAO24, ST_AsText(trajectory(atGeometry(atTime(Flight, Period), AustEnv))::GEOMETRY) AS Trajectory
FROM FlightsAust, Australia, TimePeriod
WHERE eIntersects(Flight, AustEnv) IS NOT NULL AND
    atGeometry(atTime(Flight, Period), AustEnv) IS NOT NULL;
"""

result = connection.execute(query).fetchdf()
result

,ICAO24,Trajectory
0,78921a,LINESTRING (436256.8824577683 -1340562.1823393...
1,7c7a71,LINESTRING (-1689221.693299684 -3741033.546884...
2,7c6178,LINESTRING (1525282.7155624416 -3989223.537252...
3,7c4a08,LINESTRING (-1689220.013544653 -3740714.459544...
4,7c7a85,LINESTRING (-1690036.9561117885 -3741685.60575...
...,...,...
160,7c668f,LINESTRING (394497.87011862 -3915662.848123678...
161,7c0c43,LINESTRING (-1701663.406516995 -3750856.577445...
162,7c49f8,LINESTRING (-1689319.51749081 -3740833.5400809...
163,7c8031,LINESTRING (1475548.3496748249 -3655964.416195...


In [ ]:
_ = plot_trajectories(result, 'Trajectory', crs='EPSG:3112', zoom=3)

<center><img src="figures/5.1.1.png" width=550/></center>

Flights over Australia during the whole day:

In [15]:
query = """
WITH Australia(AustEnv) AS (
    SELECT ST_MakeEnvelope(-1818009.6284, -5280319.0828, 2199539.9748, -1289403.7614)::WKB_BLOB)
SELECT ICAO24, ST_AsText(trajectory(atGeometry(Flight, AustEnv))::GEOMETRY) AS Trajectory
FROM FlightsAust, Australia
WHERE eIntersects(Flight, AustEnv) IS NOT NULL AND
    atGeometry(Flight, AustEnv) IS NOT NULL;
"""

result = connection.execute(query).fetchdf()
result

,ICAO24,Trajectory
0,7c6b4d,LINESTRING (1509269.7538551365 -4063858.980403...
1,7c1abd,LINESTRING (-1623914.4901534002 -3599357.87650...
2,7c42d5,LINESTRING (-1640920.9715498963 -3502570.59316...
3,7c61c3,LINESTRING (1884374.8401208073 -3332618.742138...
4,78921a,LINESTRING (399297.6590682664 -1289403.7613307...
...,...,...
771,7c762e,LINESTRING (1090140.8505690487 -4615342.836078...
772,7c5309,LINESTRING (1254841.06730925 -2008375.83329466...
773,7c6d22,LINESTRING (1574883.2857168755 -3932167.553320...
774,780a6e,LINESTRING (548042.6158605041 -3347065.0670057...


In [ ]:
_ = plot_trajectories(result, 'Trajectory', crs='EPSG:3112', zoom=3)

<center><img src="figures/5.1.2.png" width=550/></center>

## Query 5.2.

Compute the flights taking off in Australia between 8 am and 9 am.

In [17]:
query = """
WITH Australia(AustEnvGeom, AustEnvWKB) AS (
    SELECT ST_MakeEnvelope(-1818009.6284, -5280319.0828, 2199539.9748, -1289403.7614),
        ST_MakeEnvelope(-1818009.6284, -5280319.0828, 2199539.9748, -1289403.7614)::WKB_BLOB),
AscSpan(Span) AS ( SELECT floatspan '[1,50]' ),
TimePeriod(Period) AS (
    SELECT tstzspan '[2020-06-01 08:00:00, 2020-06-01 09:00:00)' ),
AustFlight(ICAO24, CallSign, RestFlight, RestVertRate) AS (
    SELECT ICAO24, CallSign, atTime(Flight, Period), atTime(VertRate, Period)
    FROM FlightsAust, Australia, TimePeriod
    WHERE atTime(Flight, Period) IS NOT NULL AND
        ST_Intersects(AustEnvGeom, trajectory(atTime(Flight, Period))::GEOMETRY) ),
AustFlightAscent(ICAO24, CallSign, AscTrip) AS (
    SELECT ICAO24, CallSign, atTime(RestFlight, getTime(atValues(RestVertRate, Span)))
    FROM AustFlight, AscSpan
    WHERE atValues(RestVertRate, Span) IS NOT NULL AND
        atTime(RestFlight, getTime(atValues(RestVertRate, Span))) IS NOT NULL )
SELECT ICAO24, CallSign, ST_AsText(trajectory(atGeometry(AscTrip, AustEnvWKB))::GEOMETRY) AS Geom
FROM AustFlightAscent, Australia
WHERE atGeometry(AscTrip, AustEnvWKB) IS NOT NULL;
"""

result = connection.execute(query).fetchdf()
result

,ICAO24,CallSign,Geom
0,7c7a71,YGZ,MULTILINESTRING ((-1689221.693299684 -3741033....
1,7c4a08,FD612,MULTILINESTRING ((-1689220.013544653 -3740714....
2,7c7a85,YHJ,LINESTRING (-1689873.3661743165 -3741494.76191...
3,7c6783,UTY8415,LINESTRING (-1684383.857146775 -3721252.177913...
4,7c1768,EWQ,LINESTRING (1862067.9118508697 -3244409.273758...
...,...,...,...
97,7c79b7,YBT,LINESTRING (422602.5948574225 -3925179.9326640...
98,7c6d22,VOZ9580,LINESTRING (1862458.9613407648 -3243851.227860...
99,7c49f8,FD601,LINESTRING (-1689319.51749081 -3740833.5400809...
100,7c8031,PFY9217,LINESTRING (1375831.0923983443 -3754428.857761...


In [ ]:
_ = plot_trajectories(result, 'Geom', crs='EPSG:3112', zoom=3)

<center><img src="figures/5.2.png" width=550/></center>

## Query 5.3.

Compute the flights between Auckland, Melbourne, and Sydney.

In [19]:
query = """
WITH Cities(Sydney, Auckland, Melbourne) AS (
    SELECT
        ST_MakeEnvelope(1559578.2397, -3935068.5627, 1606902.6428, -3893245.7962)::WKB_BLOB,
        ST_MakeEnvelope(3544004.0191, -4741960.5498, 3612246.2845, -4707158.270039)::WKB_BLOB,
        ST_MakeEnvelope(941409.4774, -4318032.3017, 998488.0998, -4272846.1971)::WKB_BLOB)
SELECT ICAO24, CallSign, Flight, VertRate, Velocity, GeoAltitude,
    ST_AsText(trajectory(Flight)::GEOMETRY) AS Trajectory,
    Sydney, Auckland
FROM Cities c, FlightsAust f
WHERE ( eIntersects(f.Flight, c.Sydney) AND eIntersects(f.Flight, c.Auckland) ) OR
    ( eIntersects(f.Flight, c.Melbourne) AND eIntersects(f.Flight, c.Auckland) );
"""

result = connection.execute(query).fetchdf()
result

,ICAO24,CallSign,Flight,VertRate,Velocity,GeoAltitude,Trajectory,Sydney,Auckland
0,c81d7e,ANZ1124,"[96, 118, 1, 0, 46, 3, 94, 0, 1, 0, 0, 0, 168,...","[160, 134, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 8, ...","[160, 198, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 136...","[160, 120, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 236...",LINESTRING (959879.1039420749 -4275885.4058172...,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 173, 2...","[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 106, 2..."
1,c827e3,AWK1,"[160, 4, 1, 0, 46, 3, 94, 0, 1, 0, 0, 0, 38, 1...","[160, 130, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 0, ...","[32, 139, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 17, ...","[160, 100, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 196...",LINESTRING (3574898.2648843345 -4748513.177396...,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 173, 2...","[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 106, 2..."
2,88818e,HVN68,"[128, 208, 1, 0, 46, 3, 94, 0, 1, 0, 0, 0, 15,...","[32, 229, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 197,...","[160, 251, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 242...","[160, 200, 0, 0, 33, 3, 94, 0, 1, 0, 0, 0, 140...",LINESTRING (3579272.0466909204 -4748502.980083...,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 173, 2...","[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 106, 2..."


In [ ]:
_ = plot_trajectories(result, 'Trajectory', crs='EPSG:3112', zoom=4)

<center><img src="figures/5.3.png" width=600/></center>

# Safety analysis

## Query 5.4.

Compute and display how close two aircraft have been and when.

In [21]:
query = """
WITH MinDist AS (
    SELECT atMin(f1.Flight <-> f2.Flight) AS Distance
    FROM FlightsAust f1, FlightsAust f2
    WHERE f1.ICAO24 = '7c4f1f' AND trim(f1.CallSign) = 'AE921' AND
        f2.ICAO24 = '7c777c' AND trim(f2.CallSign) = 'XVY' AND
        f1.Flight <-> f2.Flight IS NOT NULL )
SELECT startTimestamp(unnest(instants(Distance))) AS StartTime,
    getValue(unnest(instants(distance))) / 1000 AS Distance
FROM MinDist;
"""

result = connection.execute(query).fetchdf()
result

,StartTime,Distance
0,2020-06-01 02:29:24.557253+00:00,16.582328


In [22]:
query = """
WITH MinDist AS (
    SELECT f1.Flight AS f1, f2.Flight AS f2,
        atMin(f1.Flight <-> f2.Flight) AS Distance
    FROM FlightsAust f1, FlightsAust f2
    WHERE f1.ICAO24 = '7c4f1f' AND trim(f1.CallSign) = 'AE921' AND
        f2.ICAO24 = '7c777c' AND trim(f2.CallSign) = 'XVY' AND
        f1.Flight <-> f2.Flight IS NOT NULL )
SELECT ST_AsText(trajectory(MinDist.f1)::GEOMETRY) AS f1,
    ST_AsText(trajectory(MinDist.f2)::GEOMETRY) AS f2,
    startTimestamp(unnest(instants(Distance))) AS StartTime,
    getValue(unnest(instants(distance))) / 1000 AS Distance
FROM MinDist;
"""

result = connection.execute(query).fetchdf()
result

,f1,f2,StartTime,Distance
0,LINESTRING (1556871.4426050151 -3925029.497269...,LINESTRING (1575921.7314950908 -3929006.114519...,2020-06-01 02:29:24.557253+00:00,16.582328


In [23]:
tmp = result[['f1', 'f2']]
result_transposed = tmp.melt(value_vars=['f1', 'f2'], var_name='flight', value_name='geom')
result_transposed

,flight,geom
0,f1,LINESTRING (1556871.4426050151 -3925029.497269...
1,f2,LINESTRING (1575921.7314950908 -3929006.114519...


In [ ]:
_ = plot_trajectories(result_transposed, 'geom', crs='EPSG:3112', zoom=6.5)

<center><img src="figures/5.4.png" width=550/></center>

How close were the planes when they were close to the ground:

In [25]:
query = """
WITH GrdInt(ICAO24, CallSign, Interv) AS (
    SELECT ICAO24, CallSign, span(getTime(atValues(GeoAltitude, floatspan '[0, 300)'))) FROM FlightsAust
    WHERE (ICAO24 = '7c4f1f' AND trim(CallSign) = 'AE921' ) OR
        (ICAO24 = '7c777c' AND trim(CallSign) = 'XVY')),
CommonGrdInt(Interv) AS (
    SELECT f1.Interv * f2.Interv
    FROM GrdInt f1, GrdInt f2
    WHERE f1.ICAO24 < f2.ICAO24)
SELECT f1.ICAO24, f2.ICAO24,
    atMin(atTime(f1.Flight <-> f2.flight, c.Interv))::VARCHAR
FROM FlightsAust f1, FlightsAust f2, CommonGrdInt c
WHERE (f1.ICAO24 = '7c4f1f' AND trim(f1.CallSign) = 'AE921' ) AND
    (f2.ICAO24 = '7c777c' AND trim(f2.CallSign) = 'XVY') AND
    atTime(f1.Flight <-> f2.flight, Interv) IS NOT NULL;
"""

result = connection.execute(query).fetchdf()
result

,ICAO24,ICAO24_1,"CAST(atmin(attime((f1.Flight <-> f2.flight), c.Interv)) AS VARCHAR)"
0,7c4f1f,7c777c,{[16582.32811841656@2020-06-01 02:29:24.557253...


# Noise analysis

## Query 5.5.

Compute the routes of the flights taking off in Sydney and display the routes in a map.

In [26]:
query = """
WITH Sydney(SydneyEnvGeom, SydneyEnvWKB) AS (
    SELECT ST_MakeEnvelope(1559578.2397, -3935068.5627, 1606902.6428, -3893245.7962),
        ST_MakeEnvelope(1559578.2397, -3935068.5627, 1606902.6428, -3893245.7962)::WKB_BLOB),
AscSpan(Span) AS ( SELECT floatspan '[1,50]' ),
TimePeriod(Period) AS (
    SELECT tstzspan '[2020-06-01 08:00:00, 2020-06-01 09:00:00)' ),
AustFlight(ICAO24, CallSign, RestFlight, RestVertRate) AS (
    SELECT ICAO24, CallSign, atTime(Flight, Period), atTime(VertRate, Period)
    FROM FlightsAust, Sydney, TimePeriod
    WHERE atTime(Flight, Period) IS NOT NULL AND
        ST_Intersects(SydneyEnvGeom, trajectory(atTime(Flight, Period))::GEOMETRY) ),
AustFlightAscent(ICAO24, CallSign, AscTrip) AS (
    SELECT ICAO24, CallSign, atTime(RestFlight, getTime(atValues(RestVertRate, Span)))
    FROM AustFlight, AscSpan
    WHERE atValues(RestVertRate, Span) IS NOT NULL AND
        atTime(RestFlight, getTime(atValues(RestVertRate, Span))) IS NOT NULL )
SELECT ICAO24, CallSign, ST_ASText(trajectory(atGeometry(AscTrip, SydneyEnvWKB))::GEOMETRY) AS Geom
FROM AustFlightAscent, Sydney
WHERE atGeometry(AscTrip, SydneyEnvWKB) IS NOT NULL;
"""

result = connection.execute(query).fetchdf()
result

,ICAO24,CallSign,Geom
0,a782dd,UPS39,LINESTRING (1574699.9151233737 -3929463.109415...
1,a782dd,UPS34,LINESTRING (1574754.6105142268 -3930196.246073...
2,7c5323,QFA642,LINESTRING (1574757.9489381628 -3930515.952700...
3,7c617b,RSCU204,MULTILINESTRING ((1559578.2396989642 -3921431....
4,7c5323,QFA571,LINESTRING (1574679.757390244 -3929671.5250168...
5,88818e,HVN68,LINESTRING (1574798.677943643 -3931068.9494723...


In [ ]:
_ = plot_trajectories(result, 'Geom', crs='EPSG:3112', zoom=9)

<center><img src="figures/5.5.png" width=550/></center>

# Flight monitoring dashboard

In [28]:
def plot_point(df, lat_col, lon_col, color, zoom):
    fig = px.line_map(df, lat=lat_col, lon=lon_col, map_style="carto-darkmatter", zoom=zoom)
    fig.update_traces(mode='markers', marker=dict(size=5, color=color))
    fig.update_layout(margin={'r': 0, 't': 0, 'l': 0, 'b': 0})
    fig.show(renderer="notebook")

## Flight activity dashboard

All flights cruising at altitudes between 3,000 m and 8,000 m:

In [29]:
query = """
WITH TimeAltitude(Period, AltSpan) AS (
    SELECT tstzspan '[2020-06-01 08:00:00, 2020-06-01 09:00:00)', floatspan '[3000, 8000)' ),
FlightTimeSlice(ICAO24, CallSign, TripTimeSlice, AltTimeSlice) AS (
    SELECT ICAO24, CallSign, atTime(Flight, Period), atTime(GeoAltitude, Period)
    FROM Flights, TimeAltitude
    WHERE atTime(Flight, Period) IS NOT NULL ),
FlightTimeSliceCruising(ICAO24, CallSign, CruisingTrip, CruisingAltitude) AS (
    SELECT ICAO24, CallSign,
        atTime(TripTimeSlice, getTime(atValues(AltTimeSlice, AltSpan))),
        atValues(AltTimeSlice, AltSpan)
    FROM FlightTimeSlice, TimeAltitude
    WHERE atValues(AltTimeSlice, AltSpan) IS NOT NULL AND
        atTime(TripTimeSlice, getTime(atValues(AltTimeSlice, AltSpan))) IS NOT NULL ),
Instants(ICAO24, T) AS (
    SELECT ICAO24, unnest(getValues(set(timestamps(CruisingTrip)) + set(timestamps(CruisingAltitude))))
    FROM FlightTimeSliceCruising
    GROUP BY ICAO24, CruisingTrip, CruisingAltitude)
SELECT f.ICAO24, f.CallSign, getValue(atTime(CruisingAltitude, T)) AS Alt,
    ST_X(getValue(atTime(CruisingTrip, T))::geometry) AS Lon,
    ST_Y(getValue(atTime(CruisingTrip, T))::geometry) AS Lat, T FROM FlightTimeSliceCruising f, Instants i
WHERE f.ICAO24 = i.ICAO24 AND
    getValue(atTime(CruisingAltitude, T)) IS NOT NULL AND
    getValue(atTime(CruisingTrip, T)) IS NOT NULL
ORDER BY T;
"""

result = connection.execute(query).fetchdf()
print(result.shape)
result.head()

(56208, 6)


,ICAO24,CallSign,Alt,Lon,Lat,T
0,88811c,VJC768,5219.827709,106.344210,20.023373,2020-06-01 08:00:00+00:00
1,a9f123,BTQ466,3209.773328,-96.789893,29.091228,2020-06-01 08:00:00+00:00
2,ac9f51,N912NM,5130.574021,-108.043401,36.277809,2020-06-01 08:00:00+00:00
3,781023,UEA2213,7730.706477,105.857054,27.882177,2020-06-01 08:00:00+00:00
4,a809de,N617MM,6176.686883,-110.798027,35.199932,2020-06-01 08:00:00+00:00


In [ ]:
plot_point(result[:200], 'Lat', 'Lon', color='cyan', zoom=2)

<center><img src="figures/flight_activity.png" width=600/></center>

In [31]:
connection.close()